In [ ]:
# import pdfplumber
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
import pandas as pd 
from nltk.collocations import *
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [ ]:
jd='''Role - Senior Big Data Engineer
Required Technical skills - Big Data, Python, Spark, Data brick, data Spark
Desired Experience Range - 5 + years
Location of Requirement - PAN India
Desired Competencies
Must-Have
Good hands on experience in Python programming
· Data Engineering experience using AWS core services (Lambda, Glue, EMR and RedShift)
· Required skill set- SQL, Airflow · Must – Have Experience with snowflake

· Responsibility – Will be accountable for build/test complex data pipelines (batch and near real time) · Expectation – Readable documentation of all the components being develop · Experience in writing SQLs and stored procedures · Working experience with RDBMS (Oracle / Teradata)
Good-to-Have
· Good understanding of Data warehouse
· Familiarize in ETL tools like Informatica
· Experience working with NoSQL database like DynamoDB or MongoDB .'''

In [ ]:


def remove_stuff(jd):
    jd_clean = jd.replace("\xa0", "").replace("/", "").replace(".", ". ").replace("●", "")
    return jd_clean

In [ ]:
def get_cv(): #cleaned, processed, nlped cv content
    url='cv.csv'
    return pd.read_csv(url) 
# db_expander = st.beta_expander(label='Submitted resume:')
# with db_expander:
df = get_cv()

# checking data

In [ ]:
# cv data

In [ ]:
df

In [ ]:
df["All"]

In [ ]:
# job discription entered by recuriter

In [ ]:
jd

In [ ]:
def nlp(x):
    word_sent = word_tokenize(x.lower().replace("\n",""))
    _stopwords = set(stopwords.words('english') + list(punctuation)+list("●")+list('–')+list('’'))
    word_sent=[word for word in word_sent if word not in _stopwords]
    lemmatizer = WordNetLemmatizer()
    clean_text = [lemmatizer.lemmatize(word) for word in word_tokenize(" ".join(word_sent))]
#     return " ".join(NLP_Processed_CV)
    return clean_text

In [ ]:
NLP_Processed_JD=nlp(jd)


In [ ]:
NLP_Processed_JD

In [ ]:
################################################################################################################################

In [ ]:
jd_df=pd.DataFrame()
# jd_df['hi']=['I']
jd_df['jd']=[' '.join(NLP_Processed_JD)]

In [ ]:
jd_df['jd'].values[0]

In [ ]:
clean_jd=jd_df['jd'].values[0]

In [ ]:
##############################################################################################################################

In [ ]:
df["All"]

In [ ]:
cv_data=[]
for i in range(len(df["All"])):
    NLP_Processed_cv=nlp(df["All"].values[i])
    cv_data.append(NLP_Processed_cv)

In [ ]:
cv_data

In [ ]:
cv_=[]
for i in cv_data:
    cv_.append([' '.join(i)])

In [ ]:
cv_

In [ ]:
df["clean_all"]=pd.DataFrame(cv_)

# Recomendation Function

In [ ]:
df.head(1)

In [ ]:
def get_recommendation(top, df_all, scores):
    recommendation = pd.DataFrame(columns = ['name', 'degree',"email",'Unnamed: 0','mobile_number','skills','no_of_pages','score'])
    count = 0
    for i in top:
#         recommendation.at[count, 'ApplicantID'] = u
        
        recommendation.at[count, 'name'] = df['name'][i]
        recommendation.at[count, 'degree'] = df['degree'][i]
        recommendation.at[count, 'email'] = df['email'][i]
        recommendation.at[count, 'Unnamed: 0'] = df.index[i]
        recommendation.at[count, 'mobile_number'] = df['mobile_number'][i]
        recommendation.at[count, 'skills'] = df['skills'][i]
        recommendation.at[count, 'no_of_pages'] = df['no_of_pages'][i]
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    return recommendation

# cosine_similarity + TfidfVectorizer function

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


def TFIDF(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    # TF-IDF Scraped data
    tfidf_jobid = tfidf_vectorizer.fit_transform(scraped_data)

    # TF-IDF CV
    user_tfidf = tfidf_vectorizer.transform(cv)

    # Using cosine_similarity on (Scraped data) & (CV)
    cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf,x),tfidf_jobid)

    output2 = list(cos_similarity_tfidf)
    return output2
    


In [ ]:
output2 = TFIDF(df["clean_all"],jd_df['jd'])

In [ ]:
output2 

In [ ]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:100]
list_scores = [output2[i][0][0] for i in top]
TF=get_recommendation(top,df, list_scores)
TF

In [ ]:
df

# cosine_similarity + CountVectorizer


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count_vectorizer = CountVectorizer()
count_jobid = count_vectorizer.fit_transform(df["clean_all"]) #converting job data into vectors using count vectorizers
count_jobid

In [ ]:
user_count = count_vectorizer.transform(jd_df['jd'])#converting user cv data into vectors using count vectorizers
user_count

In [ ]:
print(user_count)

In [ ]:
cos_similarity_countv = map(lambda x: cosine_similarity(user_count, x),count_jobid)
output3 = list(cos_similarity_countv)
#output3

In [ ]:
output3 

In [ ]:
top = sorted(range(len(output3)), key=lambda i: output3[i], reverse=True)[:100]
list_scores = [output3[i][0][0] for i in top]
cv=get_recommendation(top, df, list_scores)
cv

# TfidfVectorizer + NearestNeighbors

In [ ]:
from sklearn.neighbors import NearestNeighbors

def KNN(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    
    KNN = NearestNeighbors(n_neighbors=7,p=2)
    KNN.fit(tfidf_vectorizer.fit_transform(scraped_data))
#     NNs = KNN.kneighbors(tfidf_vectorizer.transform(cv), return_distance=True)
    NNs = KNN.kneighbors(tfidf_vectorizer.transform(cv))
    top = NNs[1][0][1:]
    index_score = NNs[0][0][1:]

    knn = get_recommendation(top, df, index_score)
    return knn

knn = KNN(df['clean_all'], jd_df['jd'])
knn

In [ ]:

knn.to_csv('knn_cv.csv', index=False)

# Combined TFIDF, CV and KNN result together, to make a dataframe "final"

In [ ]:
knn.columns

In [ ]:
merge1 = knn[['Unnamed: 0','name', 'score']].merge(TF[['Unnamed: 0','score']], on= "Unnamed: 0")
final = merge1.merge(cv[['Unnamed: 0','score']], on = 'Unnamed: 0')
final = final.rename(columns={"score_x": "KNN", "score_y": "TF-IDF","score": "CV"})
final.head()

In [ ]:
final

In [ ]:
# Scale it
from sklearn.preprocessing import MinMaxScaler
slr = MinMaxScaler()
final[["KNN", "TF-IDF", 'CV']] = slr.fit_transform(final[["KNN", "TF-IDF", 'CV']])

# Multiply by weights
final['KNN'] = (1-final['KNN'])/3
final['TF-IDF'] = final['TF-IDF']/3
final['CV'] = final['CV']/3
final['Final'] = final['KNN']+final['TF-IDF']+final['CV']
final.sort_values(by="Final", ascending=False)

In [ ]:
# final.to_csv('final.csv', index=False)
final2 = final.sort_values(by="Final", ascending=False).copy()
final2

In [ ]:
# final2.merge(df, on="JobID")
final_df = df.merge(final2, on='Unnamed: 0')
final_df

In [ ]:
df.columns

In [ ]:
final_df.columns

In [ ]:
final_df